# Indexation Web - TP1
BERNARD Renan

## Introduction

Tout d'abord les différentes importations nécessaires pour ce Notebook.
Les fonctions utilisées sont dans le fichier __utils.py__.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import re
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import pickle

from functools import reduce

from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Indexation

### Données

Récupérons ensuite l'ensemble des données. Nous garderons un DataFrame contenons le texte, l'auteur et l'identifiant du document.

In [7]:
%%time
texts = generate_texts_dataframe()
texts.head()

CPU times: user 203 ms, sys: 75.4 ms, total: 278 ms
Wall time: 566 ms


,Text,Author,DocumentId
0,"Russia's Fuel and Energy Ministry, sitting on ...",Lynnley Browning,116673
1,Russia's Western oil joint ventures are findin...,Lynnley Browning,248885
2,Russian oil company officials said on Friday t...,Lynnley Browning,314644
3,Azerbaijan is proving more successful in attra...,Lynnley Browning,219830
4,A multinational group trying to build a $1.5 b...,Lynnley Browning,239689


Lisons quelques articles.

In [8]:
view_article(10, texts)
view_article(102, texts)


--------------------------------------------
Author : Lynnley Browning
Id : 114005
--------------------------------------------
Aluminium industry sources expressed doubt on Monday over whether producers association Kontsern Alyuminiy would be able to contain output to support sagging world prices.
"I don't believe they'll do it," said a Western trade source in Moscow.
A Moscow representative of the Krasnoyarsk smelter and two sources at the giant Bratsk plant said they had no information on plans to contain output.
"This is a very strategic question on which I have no information," said Bratsk-based Andrei Toropovsky of the plant's foreign economic relations department.
Kontsern Alyuminiy chief executive Igor Prokopov said Russian smelters had agreed to keep aluminium output at current levels and not to exploit at least 80,000-100,000 tonnes of idle annual capacity.
He and Vladimir Kalchenko, the group's first deputy chief executive, referred to weak world prices and unprofitable dom

### Préparation des textes

Le premier traitement est la transformation des phrases en un liste de mot, en miniscule, contenant uniquement des caractères numériques. Pour celà, on utilise une expression régulière.

On enlève ensuite les 'stopWords' qui n'apportent pas d'information sur le contenu de l'article.

Puis on a choisis d'utiliser un stemmer. Cette méthode enlève les préfixes et suffixes des mots pour garder uniquement la racine. Celle-ci n'est pas forcement un mot qui existe réellement.

La fonction _tokenize_text_ est aussi présente dans utils avec une meilleure documentation.

In [9]:
def tokenize_text(text):
    #Imports
    stemmer = PorterStemmer()
    stopWords = set(stopwords.words('english'))
    
    #Expression régulière. Text --> [mots]
    reg_ex = re.compile(r'[A-Za-z]+')
    bag_of_tokens = []
    text = reg_ex.findall(text.lower())
    for word in text:
        #stopWords
        if len(word) > 1 and word not in stopWords:
            #Stemmer
            bag_of_tokens.append(stemmer.stem(word))
    return bag_of_tokens

In [11]:
%%time
tokenize_text(texts.Text[0])

CPU times: user 19.8 ms, sys: 3.81 ms, total: 23.6 ms
Wall time: 23.5 ms


['russia',
 'fuel',
 'energi',
 'ministri',
 'sit',
 'remain',
 'great',
 'power',
 'wield',
 'oil',
 'sector',
 'struggl',
 'redefin',
 'new',
 'market',
 'economi',
 'energi',
 'analyst',
 'said',
 'tuesday',
 'bodi',
 'known',
 'mintopenergo',
 'could',
 'command',
 'oil',
 'well',
 'drill',
 'help',
 'restrict',
 'export',
 'quota',
 'ensur',
 'billion',
 'dollar',
 'oil',
 'export',
 'cash',
 'world',
 'third',
 'largest',
 'oil',
 'produc',
 'went',
 'kremlin',
 'coffer',
 'today',
 'power',
 'produc',
 'export',
 'plot',
 'corpor',
 'strategi',
 'larg',
 'lie',
 'hand',
 'russia',
 'newli',
 'privatis',
 'vertic',
 'integr',
 'oil',
 'firm',
 'leav',
 'ministri',
 'redefin',
 'new',
 'market',
 'economi',
 'kind',
 'use',
 'word',
 'castrat',
 'said',
 'european',
 'energi',
 'consult',
 'moscow',
 'new',
 'minist',
 'pyotr',
 'rodionov',
 'ga',
 'transport',
 'specialist',
 'mintopenergo',
 'may',
 'even',
 'less',
 'say',
 'daili',
 'busi',
 'oil',
 'firm',
 'increasingli',
 '

### Vocabulaire

On choisit d'enlever les mots qui n'apparaissent pas assez de fois dans le corpus. En effet, cela signifie qu'ils sont trop rares et qu'ils n'apportent pas assez d'informations utiles.

En gardant les mots qui apparaissent plus de 5 fois, on a un vocabulaire de 7 489 mots.

In [23]:
vocabulary = create_vocabulary(texts)

vocabulary_dict = create_vocabulary_dict(vocabulary, 5)
print(len(vocabulary_dict))
vocabulary_dict

7489


{'ab': 21,
 'abandon': 33,
 'abbey': 102,
 'abc': 13,
 'abduct': 11,
 'abf': 18,
 'abi': 12,
 'abid': 14,
 'abidjan': 76,
 'abil': 115,
 'abl': 322,
 'abn': 41,
 'abnorm': 123,
 'aboard': 22,
 'abod': 6,
 'abolish': 20,
 'abolit': 8,
 'abort': 18,
 'abound': 9,
 'abroad': 123,
 'abruptli': 6,
 'absenc': 18,
 'absent': 13,
 'absolut': 46,
 'absorb': 41,
 'abus': 35,
 'ac': 7,
 'academ': 30,
 'academi': 20,
 'acceler': 75,
 'accept': 279,
 'access': 395,
 'accid': 38,
 'accol': 10,
 'accomod': 7,
 'accompani': 32,
 'accomplish': 19,
 'accord': 459,
 'account': 582,
 'accredit': 6,
 'accret': 11,
 'accumul': 17,
 'accur': 9,
 'accus': 162,
 'ace': 7,
 'achiev': 182,
 'acid': 29,
 'acknowledg': 70,
 'acpc': 28,
 'acquir': 231,
 'acquisit': 588,
 'acquist': 8,
 'acquitt': 9,
 'acqusit': 8,
 'acr': 15,
 'across': 223,
 'act': 221,
 'action': 325,
 'activ': 529,
 'activist': 155,
 'actual': 110,
 'actuari': 9,
 'acut': 6,
 'ad': 1441,
 'adact': 6,
 'adam': 15,
 'adapt': 24,
 'add': 143,
 'add

### Création de l'Index

Afin de trouver rapidement les documents qui contiennent un certain mot, il a fallu créer un index. On a choisit de faire un index de la forme suivante : \\
    {"token1" : {"total_occurences" : 4, \
                         article1 : {"locations" : [12, 13, 14],
                                 "occurences" : 3}, \
                         article2 : {"locations" : [9],
                                 "occurences" : 1}}}
                                 
Nous avons d'abord essayer de créer l'index avec une programmation simple comprenant différentes boucles.    --> temps ? pour comparaisons                         
                                 
L'index est crée selon les specifications de la fonction __create_index_from_text(...)__ dont les explications sont disponibles dans __utils.py__. Pour accélerer le calcul, nous utilisons le modèle _MapReduce_. Le __mapper__ ici correspond à la création de l'index pour un seul document, le __reducer__ correspond à l'_addition_ des index obtenus.

La fonction  __create_index_from_text(...)__  s'utilise sur les textes brutes car la fonction de préparation du texte est utilisée à l'intérieur de notre fonction.



In [31]:
%%time
index_simple = create_index(vocabulary_dict, texts)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


TypeError: 'RangeIndex' object is not callable

In [13]:
%%time 
index = reduce(sum_two_indexes, list(map(lambda x : create_index_from_text(texts.Text[x], x), range(len(texts)))))

CPU times: user 31 s, sys: 149 ms, total: 31.1 s
Wall time: 31.1 s


## Requêtes


## Ranking

### Création de la matrice TF-IDF

La matrice TF_IDF (la représation du corpus dans l'espace TF-IDF) est également créer suivant un modèle _MapReduce_, le __mapper__ étant le calcul du TF-IDF pour chaque documents d'un _token_, le __reducer__ étant une simple concaténation.

In [14]:
%%time
tokens_count = generate_tokens_count(index)
matrix_tfidf = np.concatenate(list(map(lambda token: calculate_tf_idf_for_token(index[token], tokens_count), list(index.keys()))), axis=1)

CPU times: user 2.49 s, sys: 132 ms, total: 2.63 s
Wall time: 2.62 s


### Mise en place d'une requête du corpus

Pour requêter (_query_) le corpus, nous calculons simplement le produit scalaire de la représentation du texte de la requête dans l'espace TF-IDF par la matrice. Le calcul du TF-IDF se fait comme si la requête était un nouvel article du corpus.

In [15]:
%%time
query_corpus("british dairy", index, matrix_tfidf, texts, nb_to_show=20)

CPU times: user 105 ms, sys: 3.94 ms, total: 109 ms
Wall time: 94 ms


,DotProduct,Text,Author,DocumentId
2010,0.321888,British food producer Northern Foods Plc poste...,Tim Farrand,202017
2014,0.238622,Britain's Northern Foods sees world dairy comm...,Tim Farrand,204606
2047,0.194796,British dairy and distribution company Unigate...,Tim Farrand,201017
2030,0.121621,British dairy and distribution company Unigate...,Tim Farrand,199222
2338,0.062595,Dairy and fruit juice group National Foods Ltd...,Bernard Hickey,30677
1848,0.043480,Britain's Home Office has warned ethnic minori...,Tan Ee Lyn,380733
1146,0.042589,British Telecom Sunday formally announced its ...,Kirstin Ridley,162195
2398,0.040190,The three-year-old partnership between British...,Robin Sidel,144598
1153,0.039991,Are you a businessman looking for generous tax...,Jane Macartney,105215
2471,0.039762,British Telecom and MCI Communications Sunday ...,Nick Louth,162521
